In [ ]:
!pip install fastapi pyngrok uvicorn
!pip install nest_asyncio
!pip install tesseract
# SUDO APT INSTALL FOR LINUX
!sudo apt install tesseract-ocr
!pip install pytesseract
!pip install python-multipart
!pip install pytesseract
!pip install python-multipart
pip install textstat

In [14]:
import numpy as np
import sys, os, setuptools, tokenize
from fastapi import FastAPI, UploadFile, File
from fastapi.middleware.cors import CORSMiddleware
from starlette.requests import Request
import io
import cv2
import pytesseract
from pydantic import BaseModel
import nest_asyncio
import textstat.textstat
from google.colab.patches import cv2_imshow
nest_asyncio.apply()

def read_img(img):
  pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'
  text = pytesseract.image_to_string(img)
  return(text)
 
app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

class ImageType(BaseModel):
  url: str

@app.post("/predict/") 
def prediction(request: Request, 
  file: bytes = File(...)):
  # If we're dealing with a post request
  if request.method == "POST":
    image_stream = io.BytesIO(file)
    image_stream.seek(0)
    file_bytes = np.asarray(bytearray(image_stream.read()), dtype=np.uint8)
    frame = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR)

    # blur
    blur = cv2.GaussianBlur(frame, (3,3), 0)

    # convert to hsv and get saturation channel
    sat = cv2.cvtColor(blur, cv2.COLOR_BGR2HSV)[:,:,1]

    # threshold saturation channel
    thresh = cv2.threshold(sat, 50, 255, cv2.THRESH_BINARY)[1]

    # apply morphology close and open to make mask
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (9,9))
    morph = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=1)
    mask = cv2.morphologyEx(morph, cv2.MORPH_OPEN, kernel, iterations=1)

    # do OTSU threshold to get circuit image
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    otsu = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]

    # write black to otsu image where mask is black
    otsu_result = otsu.copy()
    otsu_result[mask==0] = 0


    # Now we use pytesseract to extract the text
    label = read_img(otsu)

    sample_story = label
    cv2_imshow(otsu)
    return textstat.textstat.flesch_kincaid_grade(sample_story)

In [ ]:
from pyngrok import ngrok
import uvicorn

url = ngrok.connect(port=8000)
print('Public URL:', url)
uvicorn.run(app, port=8000)